In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

In [4]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [5]:
train["text_plus_keyword"] = train["keyword"].fillna("") + " " + train["text"]
test["text_plus_keyword"] = test["keyword"].fillna("") + " " + test["text"]

In [6]:
train

,id,keyword,location,text,target,text_plus_keyword
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,Two giant cranes holding a bridge collapse in...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,@aria_ahrary @TheTawniest The out of control ...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. ht...
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,Police investigating after an e-bike collided...


In [7]:
test

,id,keyword,location,text,text_plus_keyword
0,0,NaN,NaN,Just happened a terrible car crash,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","Heard about #earthquake is different cities, ..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","there is a forest fire at spot pond, geese ar..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FAST...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,Storm in RI worse than last hurricane. My cit...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,Green Line derailment in Chicago http://t.co/...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,MEG issues Hazardous Weather Outlook (HWO) ht...


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 7613 non-null   int64 
 1   keyword            7552 non-null   object
 2   location           5080 non-null   object
 3   text               7613 non-null   object
 4   target             7613 non-null   int64 
 5   text_plus_keyword  7613 non-null   object
dtypes: int64(2), object(4)
memory usage: 357.0+ KB


In [9]:
train['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [10]:
train.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,7613.0,NaN,NaN,NaN,5441.934848,3137.11609,1.0,2734.0,5408.0,8146.0,10873.0
keyword,7552,221,fatalities,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location,5080,3341,USA,104,NaN,NaN,NaN,NaN,NaN,NaN,NaN
text,7613,7503,11-Year-Old Boy Charged With Manslaughter of T...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target,7613.0,NaN,NaN,NaN,0.42966,0.49506,0.0,0.0,0.0,1.0,1.0
text_plus_keyword,7613,7526,fatal 11-Year-Old Boy Charged With Manslaughte...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train.isnull().sum()

id                      0
keyword                61
location             2533
text                    0
target                  0
text_plus_keyword       0
dtype: int64

In [12]:
train['keyword'] = train['keyword'].fillna('Unknown')
train['location'] = train['location'].fillna('Unknown')
test['keyword'] = test['keyword'].fillna('Unknown')
test['location'] = test['location'].fillna('Unknown')

In [13]:
train.isnull().sum()

id                   0
keyword              0
location             0
text                 0
target               0
text_plus_keyword    0
dtype: int64

In [14]:
train['keyword'].value_counts()

keyword
Unknown                  61
fatalities               45
armageddon               42
deluge                   42
harm                     41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 222, dtype: int64

In [15]:
train.head(20)

,id,keyword,location,text,target,text_plus_keyword
0,1,Unknown,Unknown,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...
1,4,Unknown,Unknown,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada
2,5,Unknown,Unknown,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...
3,6,Unknown,Unknown,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o..."
4,7,Unknown,Unknown,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...
5,8,Unknown,Unknown,#RockyFire Update => California Hwy. 20 closed...,1,#RockyFire Update => California Hwy. 20 close...
6,10,Unknown,Unknown,#flood #disaster Heavy rain causes flash flood...,1,#flood #disaster Heavy rain causes flash floo...
7,13,Unknown,Unknown,I'm on top of the hill and I can see a fire in...,1,I'm on top of the hill and I can see a fire i...
8,14,Unknown,Unknown,There's an emergency evacuation happening now ...,1,There's an emergency evacuation happening now...
9,15,Unknown,Unknown,I'm afraid that the tornado is coming to our a...,1,I'm afraid that the tornado is coming to our ...


In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [17]:
nltk.download('stopwords')
nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/

In [18]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Define a function for preprocessing
def preprocess_text(text):
    # Lowercase text
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Remove punctuations and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization and remove stopwords
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

# Apply preprocessing to train and test datasets
train['clean_text'] = train['text_plus_keyword'].apply(preprocess_text)
test['clean_text'] = test['text_plus_keyword'].apply(preprocess_text)

In [19]:
train.head(20)

,id,keyword,location,text,target,text_plus_keyword,clean_text
0,1,Unknown,Unknown,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...,deed reason earthquake may allah forgive u
1,4,Unknown,Unknown,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,5,Unknown,Unknown,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...,resident asked shelter place notified officer ...
3,6,Unknown,Unknown,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o...",people receive wildfire evacuation order calif...
4,7,Unknown,Unknown,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...,got sent photo ruby alaska smoke wildfire pour...
5,8,Unknown,Unknown,#RockyFire Update => California Hwy. 20 closed...,1,#RockyFire Update => California Hwy. 20 close...,rockyfire update california hwy closed directi...
6,10,Unknown,Unknown,#flood #disaster Heavy rain causes flash flood...,1,#flood #disaster Heavy rain causes flash floo...,flood disaster heavy rain cause flash flooding...
7,13,Unknown,Unknown,I'm on top of the hill and I can see a fire in...,1,I'm on top of the hill and I can see a fire i...,im top hill see fire wood
8,14,Unknown,Unknown,There's an emergency evacuation happening now ...,1,There's an emergency evacuation happening now...,there emergency evacuation happening building ...
9,15,Unknown,Unknown,I'm afraid that the tornado is coming to our a...,1,I'm afraid that the tornado is coming to our ...,im afraid tornado coming area


In [20]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,AutoConfig
from transformers import Trainer, TrainingArguments
import torch
from datasets import Dataset
import torch.nn as nn

In [21]:
train = train.rename(columns={"target": "labels"})

In [22]:
train.head()

,id,keyword,location,text,labels,text_plus_keyword,clean_text
0,1,Unknown,Unknown,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake ...,deed reason earthquake may allah forgive u
1,4,Unknown,Unknown,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,5,Unknown,Unknown,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are...,resident asked shelter place notified officer ...
3,6,Unknown,Unknown,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation o...",people receive wildfire evacuation order calif...
4,7,Unknown,Unknown,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as...,got sent photo ruby alaska smoke wildfire pour...


In [23]:
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [24]:
num_0 = 4342
num_1 = 3271
total = num_0 + num_1

# Inverse frequency weighting
weight_0 = total / (2 * num_0)
weight_1 = total / (2 * num_1)

class_weights = torch.tensor([weight_0, weight_1])
print("Class weights:", class_weights)

class WeightedDeberta(AutoModelForSequenceClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        self.class_weights = class_weights

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss(weight=self.class_weights.to(labels.device))
            loss = loss_fn(outputs.logits, labels)
            return {"loss": loss, "logits": outputs.logits}
        return outputs

Class weights: tensor([0.8767, 1.1637])


In [25]:
model_name = "microsoft/deberta-v3-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, num_labels=2)

model = WeightedDeberta.from_pretrained(
    model_name,
    config=config,
)
model.class_weights = torch.tensor([0.8767, 1.1637])

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# model_name = "microsoft/deberta-v3-base" 
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
def tokenize(example):
    return tokenizer(example['clean_text'], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/7613 [00:00<?, ? examples/s]

In [28]:
train_dataset = train_dataset.remove_columns(["id", "text", "location", "keyword", "clean_text","text_plus_keyword"])

In [29]:
train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7613
})

In [30]:
from sklearn.model_selection import train_test_split
train_val = train_dataset.train_test_split(test_size=0.3, seed=42)
train_ds = train_val["train"]
val_ds = train_val["test"]

In [31]:
train_ds

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5329
})

In [32]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    score = f1_score(labels, predictions, average="weighted")
    
    return {"f1": score, "accuracy": accuracy}

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to GPU

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-5): 6 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [35]:
print("CUDA available:", torch.cuda.is_available())
print("GPU device name:", torch.cuda.get_device_name(0))
print("Model device:", next(model.parameters()).device)

CUDA available: True
GPU device name: Tesla T4
Model device: cuda:0


In [36]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    evaluation_strategy="epoch",
    logging_first_step=True,
    save_strategy="no", 
    logging_steps=10,
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [37]:
trainer= Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics = compute_metrics,
)

In [38]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.456400,0.431901,0.822757,0.824431


TrainOutput(global_step=334, training_loss=0.47473727532489574, metrics={'train_runtime': 149.4654, 'train_samples_per_second': 35.654, 'train_steps_per_second': 2.235, 'total_flos': 352971956407296.0, 'train_loss': 0.47473727532489574, 'epoch': 1.0})

In [ ]:
# metrics = trainer.evaluate()
# print(metrics)

In [39]:
test_dataset = test_dataset.map(tokenize, batched=True)

# Remove the same columns to avoid inference errors
test_dataset = test_dataset.remove_columns(["id", "text", "location", "keyword", "clean_text", "text_plus_keyword"])

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [40]:
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [41]:
df_test_results = pd.DataFrame({'id': test['id'], 'target': predicted_labels})

In [42]:
df_test_results.to_csv('submission.csv', index=False)

In [43]:
from IPython.display import FileLink

# Path to your file
file_path = 'submission.csv'

# Generate and display the download link
FileLink(file_path)

/kaggle/working/submission.csv